# RBM Deep Dive with Tensorflow 

In this notebook we provide a complete walkthough of the Restricted Boltzmann Machine (RBM) algorithm with applications to recommender systems. In particular, we use as a case study the [movielens dataset](https://movielens.org), comprising the ranking of movies (from 1 to 5) given by viewers. A quickstart version of this notebook can be found [here](notebooks/00_quick_start/rbm_movielens.ipynb).  

### Overview 

The RBM is a generative neural network model used to perform unsupervised learning. The main task of an RBM is to learn the joint probability distribution $P(v,h)$, where $v$ are the visible units and $h$ the hidden ones. The hidden units represent latent variables while the visible units are clamped on the input data. Once the joint distribution is learnt, new examples are generated by sampling from it.  

The implementation presented here is based on the article by Ruslan Salakhutdinov, Andriy Mnih and Geoffrey Hinton [Restricted Boltzmann Machines for Collaborative Filtering](https://www.cs.toronto.edu/~rsalakhu/papers/rbmcf.pdf) with the exception that here we use multinomial units instead of one-hot encoded. 

### Advantages of RBM: 

The model generates ratings for a user/movie pair using a collaborative filtering based approach. While matrix factorization methods learn how to reproduce an instance of the user/item affinity matrix, the RBM learns its underlying probability distribution. This has several advantages: 

- Generalizability : the model generalize well to new examples as long as they do not differ much in probability
- Stability in time: if the recommendation task is time-stationary, the model does not need to be trained often to accomodate new ratings/users. 
- Scales well with the size of the dataset and the sparsity of the user/affinity matrix. 
- The tensorflow implementation presented here allows fast, scalable  training on GPU 

### Outline 

This notebook is organizaed as follows:

1. RBM Theory 
2. Tensorflow implementation of the RBM 
3. Data preparation and inspection
4. Model application, performance and analysis of the results  

Sections 1 and 2 requires basic knowledge of linear algebra, probability theory and tensorflow while  
sections 3 and 4 only requires some basic data science understanding. **Feel free to jumpo to the section you are most interested in!**



## 0 Global Settings and Import

In [ ]:
from __future__ import print_function
from __future__ import absolute_import
from __future__ import division

# set the environment path to find Recommenders
import sys
sys.path.append("../../")

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import papermill as pm
from zipfile import ZipFile

from reco_utils.recommender.rbm.Mrbm_tensorflow import RBM
from reco_utils.dataset.rbm_splitters import splitter

from reco_utils.dataset.url_utils import maybe_download
from reco_utils.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k

#For interactive mode only
%load_ext autoreload
%autoreload 2

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))

# 1. RBM Theory 

## 1.1 Overview and main differences with other recommender algorithms

A Restricted Boltzmann Machine (RBM) is an undirected graphical model with origin in the statistical mechanics (or physics) of magnetic systems. Statistical mechanics (SM) provides a probabilistic description of complex systems made of a huge number of constituents (typically $\sim 10^{23}$); instead of looking at a particular instance of the system, the aim of SM is to describe their **typical** behaviour. This approach has been succesfull for the description of gases, liquids, complex materials (e.g. semiconductors) and even the famous [Higgs boson](https://en.wikipedia.org/wiki/Higgs_boson)!

Being designed to handle and organize a large amount of data, SM finds ideal applications in modern learnign algorithms. In the context of **recommender systems**, the idea is to learn typical user behaviour instead of particular instances. To understand this better, consider the most general setup of a recmmendation problem: there are $m$ users rating $n$ items according to some scale (e.g. 1 to 5). In a typical scenario of online shopping, streaming services or decision processes, the user only rates a subset $l \ll m$ of the products. If we now create a matrix representation of this problem, we obtain the user/item affinity matrix $X$. In a more readable table form, $X$ will look like this:

|     |$i_1$  |$i_2$  |$i_3$  |  ... |$i_m$  | 
|-----|-------|-------|-------|------|-------|
|$u_1$|5      |0      |2      |0 ... |1      |
|$u_2$|0      |0      |3      |4 ... |0      |
|...  |...    |...    |...    |...   |...    |
|$u_m$|3      |3      |0      |5...  |2      |

where zeroes denote unrated items. In a nutshell, the recommender task is to "fill in" the missing ratings (later we will see that in practice this is not the only criteria to recommend a product). The classical approach to this problem is called matrix factorization: the basic idea is to decompose $X$ into a user ($P$) and item ($Q$) matrix, such that $X = Q^T P$. The dimensions of the two matrices are $dim(Q) = (f, n)$ and $dim(P)= (f,m)$ where $f \le m,n$ is the number of latent factors, e.g. the genre of a movie, the type of food etc... and it is an hyperparameter of the model, for more details see the [ALS notebook](notebooks/02_modeling/als_deep_dive.ipynb). By learning $Q$ and $P$ we try to reproduce a particular instance of $X$ (provided by the available data) and use this information to fill up the missing matrix elements. 

The RBM approach is to look at $X$ as a particular realization (sample) of a more general process; instead of learning a specific $X$, we try to learn the matrix distrbution from which $X$ has been sampled form. Efectively, we learn the typical distirbution of *tastes* (i.e. latent factors) and use this information to *generate* new ratings. For this reaoson, this class of neural network models is also called **generative**. Consider the following example: imagine you are given the  income distribution per age window of a particular country (this is easy to find from goverments data), then we could fix the age window and *generate* virtual citizens with various incomes by sampling from this distibution.   

## 1.2 Model 

The central quantity of every SM model is the [Boltzmann distribution ](https://en.wikipedia.org/wiki/Boltzmann_distribution); this can be seen as the least biased probability distribution on a certain probability space $\Sigma$ and can be obtained using a maximum entropy principle on the space of distributions over $\Sigma$. Its typical form is: 

$P = \frac{1}{Z} \, e^{- \beta \, H}$, 

where $Z$ is a normalization constant known as the partition function, $\beta$ is a noise parameter with units of inverse energy and $H$ is the Hamiltonian, or energy function of the system. For this reason, this class of models is also known as *energy based* in computer science. In physics, $\beta$ is the inverse temperature of the system in units of Boltzmann's constant, but here we will effectively rescale inside $H$, so that this is now a pure number. $H$ describes the behaviour of two sets of stochastic vectors, typically called $v_i$ (visibles) and $h_j$ (hidden). The former constitutes both the input *and* the ouput of the algo (this will be clear later), while the hidden units are the latent factors we want to learn. This structure results in the following Neural Network topology





# 3 Data preparation and inspection 

The Movielens dataset comes in different sizes, denoting the number of available ratings. The number of users and rated movies also changes across the different dataset. The data are imported in a pandas dataframe including the **user ID**, the **item ID**, the **ratings** and a **timestamp** denoting when a particular user rated a particular item. Although this last feature could be explicitely included, it will not be considered here. The underlying assumption of this choice is that user's tastes are weakly time dependent, i.e. a user's taste typically chage on time scales (usually years) much longer than the typical recommendation time scale (e.g. hours/days). As a consequence, the joint probability distribution we want to learn can be safely considered as time dependent. Nevertheless, timestamps could be used as *contextual variables*, e.g. recommend a certain movie during the weekend and another during weekdays.  

